In [ ]:
import pandas as pd
from breeze_connect import BreezeConnect
import mysql.connector as sqlConnector
from datetime import date
import time
global session
session=''
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class DBBasic:
    con = sqlConnector.connect(host="xyz.in", user="user", passwd="*******", database="algo", port="3309")
    def __init__(self):
        pass
    
    def GetTempToken(self):
        cursor = self.con.cursor()
        cursor.execute('Select SQL_NO_CACHE Token, tstamp from icicitemptoken ORDER BY ID DESC LIMIT 1;')
        for row in cursor:
            return row
        

In [ ]:
api_key = "7N342890R6384pOG66940#Y4@72BU42y"
apisecret = "l~1W24M2705L45653KVj8013#7%03*F7"
db = DBBasic()
breeze = BreezeConnect(api_key=api_key)

In [ ]:
def InitiateICICI():
    tempToken = db.GetTempToken()
    if tempToken is None:
        print('tempToken is None:')
        print('Login to API and rerun the program')
        return False
    ttoken = tempToken[0]
    global session
    session = ttoken
    try:
        breeze.generate_session(api_secret=apisecret, session_token=ttoken)
        return True
    except Exception as e:
        print(e)
        return False

In [ ]:
InitiateICICI()

In [ ]:
def GetLTP(stock_code):    
    data = breeze.get_quotes(stock_code=stock_code,
                        exchange_code="NSE",
                        product_type="cash")
    put_data = pd.DataFrame(data["Success"])
    if put_data.empty:
        print('Data not found:')
        df = pd.DataFrame()
        return df
    else:
        df = put_data[['ltp']]
    return df['ltp'].iloc[0]

In [ ]:
GetLTP('TCS')

In [ ]:
conTick = sqlConnector.connect(host="localhost", user="root", passwd="Airforce*123", database="algo", port="3306")
def getMonitorStocks():
    query = "SELECT CONCAT(shortName) as instrument_token, price, targetprice FROM kimalgo_test.MonitorStocks where enabled = 1"
    df = pd.read_sql(query, con=conTick)
    return df


In [ ]:
from pygame import mixer
def monitorStock():
    print('Monitoring Price!!!!!!')
    print('\a')
    dfStocks = getMonitorStocks()
    listStocks = dfStocks['instrument_token'].tolist()
    for stock in listStocks:
        cprice = GetLTP(stock)
        rslt_df = dfStocks.loc[(dfStocks['instrument_token'] == stock)]
        reqPrice = rslt_df.iloc[0]['price'] 
        targetPrice = rslt_df.iloc[0]['targetprice']
        if cprice <= reqPrice or cprice >= targetPrice:
            print('Price Alert: ' + stock + ' price: ' + str(cprice))
            mixer.init() 
            sound=mixer.Sound("alarm.wav")
            sound.play()

In [ ]:
while True:
    monitorStock()
    time.sleep(60)

In [ ]:
scrip = pd.read_csv("NSEScripMaster.txt")
scrip = scrip[[' "ShortName"',' "CompanyName"', ' "ExchangeCode"']]
columnsname1 = [' "ShortName"',' "CompanyName"', ' "ExchangeCode"']
columnsname2 = ['ShortName','CompanyName', 'ExchangeCode']
rename_columns = dict(zip(columnsname1,columnsname2))
scrip.rename(columns=rename_columns, inplace=True)
scrip = scrip[scrip.ExchangeCode == 'HDFCBANK']
scrip.head()